In [1]:
# Outside imports
import os
import importlib
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [3]:

import model
import train
import evaluate
import train_util
import data_util.data
import data_util.batcher
import data_util.config

importlib.reload(model)
importlib.reload(train)
importlib.reload(evaluate)
importlib.reload(train_util)
importlib.reload(data_util.config)
importlib.reload(data_util.data)
importlib.reload(data_util.batcher)

from evaluate import *
from model import *
from train import *
from train_util import *
from data_util.data import *
from data_util.batcher import *


NameError: name 'importlib' is not defined

In [49]:
import spacy

In [32]:
# load real data
data_path = '/Users/rowancassius/Desktop/capstone/data/context_task_data.tsv'
dat = pd.read_csv(data_path, sep='\t')

# fill nas
dat.fillna('', inplace=True)

# train/test split
np.random.seed(111)
dat = dat.sample(frac=1)
train_size = int(.8*dat.shape[0])
train_data = dat[:train_size]
test_data = dat[train_size:]

# fit tokenizer
FILTERS = '!"&()*+,:;<=>?[\\]^_`{|}~\t\n'
tokenizer = Tokenizer(lower=True, filters=FILTERS)
tokenizer.fit_on_texts(train_data.Context.values + train_data.TaskSentence.values + train_data.Summary.values)
tokenizer.texts_to_texts = lambda texts: tokenizer.sequences_to_texts(tokenizer.texts_to_sequences(texts))

# transform texts
def prep_data(data):
    for c in ['Context', 'TaskSentence', 'Summary']:
        data[c] = tokenizer.texts_to_texts(data[c])
    return data

# train_data = prep_data(train_data)
# test_data = prep_data(test_data)

In [4]:
vocab = data.Vocab(
    words = tokenizer.word_index.keys(), 
    max_size=len(tokenizer.word_index))

max_size of vocab was specified as 7632; we now have 7632 words. Stopping reading.
Finished constructing vocabulary of 7632 total words. Last word added: refs


In [5]:
task_batcher = TaskBatcher(
    examples=train_data.to_dict('records')[:10],
    vocab=vocab,
    mode='train',
    batch_size=32,
    single_pass=False
)

In [6]:
random.seed(123)
T.manual_seed(123)
if T.cuda.is_available():
    T.cuda.manual_seed_all(123)
    
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

In [7]:
train_mle = "yes"
train_rl = "no"
mle_weight = 1.0
load_model = None
new_lr = None
rl_weight = 1 - mle_weight

opt = Namespace(train_mle = train_mle, 
                train_rl = train_rl, 
                mle_weight = mle_weight, 
                load_model = load_model,
                new_lr = new_lr, 
                rl_weight = rl_weight)


train_processor = TaskTrain(vocab, task_batcher, opt)

In [8]:
config.save_model_path = "data/lstm_seg_saved_models"

train_processor.trainIters(n_iters=1000, model=TaskModel, report_every=1, save_every = 50)

iter: 1 mle_loss: 5.408 reward: 0.0000
iter: 2 mle_loss: 5.484 reward: 0.0000
iter: 3 mle_loss: 5.277 reward: 0.0000
iter: 4 mle_loss: 5.264 reward: 0.0000
iter: 5 mle_loss: 5.171 reward: 0.0000
iter: 6 mle_loss: 5.139 reward: 0.0000
iter: 7 mle_loss: 4.869 reward: 0.0000
iter: 8 mle_loss: 4.024 reward: 0.0000
iter: 9 mle_loss: 3.727 reward: 0.0000
iter: 10 mle_loss: 3.362 reward: 0.0000
iter: 11 mle_loss: 3.222 reward: 0.0000
iter: 12 mle_loss: 3.280 reward: 0.0000
iter: 13 mle_loss: 3.115 reward: 0.0000
iter: 14 mle_loss: 3.010 reward: 0.0000
iter: 15 mle_loss: 2.945 reward: 0.0000
iter: 16 mle_loss: 2.895 reward: 0.0000
iter: 17 mle_loss: 2.869 reward: 0.0000
iter: 18 mle_loss: 2.718 reward: 0.0000
iter: 19 mle_loss: 2.643 reward: 0.0000
iter: 20 mle_loss: 2.559 reward: 0.0000
iter: 21 mle_loss: 2.433 reward: 0.0000
iter: 22 mle_loss: 2.395 reward: 0.0000
iter: 23 mle_loss: 2.401 reward: 0.0000
iter: 24 mle_loss: 2.294 reward: 0.0000
iter: 25 mle_loss: 2.430 reward: 0.0000
iter: 26 

RuntimeError: invalid multinomial distribution (encountering probability entry < 0)

Decoding Time

In [24]:
task = "validate"

load_model = os.path.join(config.log_root, "data/lstm_seg_saved_models/0000050.tar") # model directory

opt = Namespace(task = task, load_model = load_model) # opt

vocab = data.Vocab( # vcocab obj
    words = tokenizer.word_index.keys(), 
    max_size=len(tokenizer.word_index))

# new batcher for evaluation
task_batcher = TaskBatcher( # Batching obj
    examples=test_data.to_dict('records')[:20],
    vocab=vocab, 
    mode='train', 
    batch_size=10, 
    single_pass=True)

eval_processor = TaskEvaluate(vocab, task_batcher, opt) # Evaluation object

example_generator completed reading all examples. No more data.


Exception in thread Thread-18:
Traceback (most recent call last):
  File "/Users/rowancassius/Desktop/capstone/LSTM_Summarizer/data_util/batcher.py", line 390, in text_generator
    example = next(example_generator)
StopIteration

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/rowancassius/opt/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/Users/rowancassius/opt/anaconda3/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/rowancassius/Desktop/capstone/LSTM_Summarizer/data_util/batcher.py", line 371, in fill_example_queue
    context, task, summary = next(input_gen) # read the next example from file. article and abstract are both strings.
RuntimeError: generator raised StopIteration



In [25]:
decoded_sents, ref_sents = eval_processor.evaluate_batch(model=TaskModel)

torch.Size([10, 25, 512])
Summarizing Batch...
torch.Size([10, 25, 512])
Summarizing Batch...
INFO:tensorflow:Finished reading dataset in single_pass mode.


In [ ]:
class Predictor:
    def __init__(model_path, vocab, opt):
        pass
    
    def predict(examples):
        batch_size = len(examples)
        batcher = Batcher(...)
        

In [29]:
pd.set_option('display.max_colwidth', -1)

In [30]:
pd.DataFrame({'ref': ref_sents, 'decoded': decoded_sents})

,ref,decoded
0,call sender at,call me at
1,save this message,save message with your your new
2,change password,user change change change from
3,rsvp to sender about meeting by friday may 25,return to to your response
4,look into list of commercial teams within ena and tell sender who to allocate to,look at file
5,check with your people.,you with your people
6,call sender,if to this give
7,review and give,review and know
8,calendar this and make a file,calendar calendar file
9,bring the down,go ahead down down


In [33]:
test_data.head()

,#,Task,Context,TaskSentence,Summary,Labeler,NoRequestInContext,Urgent,NotRequest,Unsure/Discuss,RandomNumber
1046,5362,Go ahead and bring the Yates,"Do you mean 12,000 at 6789?. If so that's fine.",Go ahead and bring the Yates down also.,Bring the Yates down,Natalie,1,0,0,,0.312529
1105,5421,Please call me at X30649.,"Vince, if there are some differences that we need to correct, you and I can meet tomorrow when you bring the contract in.. I am available anytime after lunch.",Please call me at X30649.,Call SENDER at X30649,Natalie,1,0,0,,0.239826
392,393,"TWanda, please calendar and then make a file entitled Employees in Right",,"TWanda, please calendar and then make a file entitled Employees in Right Company and give to me on the date of the meeting.",Calendar this and make a file,Rowan,,0,,,0.274254
276,277,Please save this message with your new registered email address for future reference.,Dear plucci01. We received your request and have successfully changed your registered email address at Match.com.,Please save this message with your new registered email address for future reference.,Save this message,Rowan,1,0,,,0.645821
494,495,Password: emclaug Please Change,"Your PRODUCTION User ID and Password has been set up on UBSWenergy.. Please follow the steps below to access the new environment:. From Internet Explorer connect to the UBSWenergy Production Cluster through the following link: http://remoteservices.netco.enron.com/ica/ubswenergy.ica use your UBSWenergy/Enron NT Log In & Password. From the second Start menu, select appropriate application:",User ID: emclaug Password: emclaug Please Change,Change password,Rowan,,0,,,0.098290


In [31]:
test_data

,#,Task,Context,TaskSentence,Summary,Labeler,NoRequestInContext,Urgent,NotRequest,Unsure/Discuss,RandomNumber
1046,5362,Go ahead and bring the Yates,do you mean 12 000 at . if so that's,go ahead and bring the down,bring the down,Natalie,1,0,0,,0.312529
1105,5421,Please call me at X30649.,vince if there are some that we need to correct you and i can meet tomorrow when you bring the contract in.. i am available after,please call me at,call sender at,Natalie,1,0,0,,0.239826
392,393,"TWanda, please calendar and then make a file entitled Employees in Right",,twanda please calendar and then make a file entitled employees in right company and give to me on the date of the meeting.,calendar this and make a file,Rowan,,0,,,0.274254
276,277,Please save this message with your new registered email address for future reference.,dear we received your request and have successfully changed your registered email address at,please save this message with your new registered email address for future,save this message,Rowan,1,0,,,0.645821
494,495,Password: emclaug Please Change,your production user id and password has been set up on please follow the steps below to access the new . from internet explorer to the production through the following link http use your log in from the second start select appropriate application,user id password please change,change password,Rowan,,0,,,0.098290
...,...,...,...,...,...,...,...,...,...,...,...
118,119,Please review and let's discuss.,barry attached is a draft of the confirm for the deal.. this confirm reflect only the volumes related to package the confirm for package one would be to this one with term and volumes as we discussed this confirm does not reflect the revisions needed to the existing,please review and let's,review draft of confirm for deal,Rowan,1,,,,0.096584
681,3448,Please override POP'S daily volume presently zero to reflect daily,this is a follow up to the note i gave you on monday preliminary flow data provided by daren .,please daily volume zero to reflect daily activity you can obtain from gas control.,daily,George,,,,,0.555117
86,87,"Everyone, pls forward to me ASAP your list of top customers you want invited",,everyone pls forward to me asap your list of top customers you want invited to the oct.,forward top customer list to sender,Rowan,,,,,0.441849
724,3491,Please print the email and attachment click view .,,please print the email and attachment click view .,print email and,George,,,,,0.024133


In [27]:
ref_sents

['call sender at',
 'save this message',
 'change password',
 'rsvp to sender about meeting by friday may 25',
 'look into list of commercial teams within ena and tell sender who to allocate to',
 'check with your people.',
 'call sender',
 'review and give',
 'calendar this and make a file',
 'bring the down',
 'sort book id by risk type',
 'call sender at on changes',
 'reply to sender by 10 a.m. on wednesday march',
 'pay by cash or check',
 'correct problem why is sender receiving copies of responses to this group .',
 'on bid forms to bottom of page',
 "confirm sender's opinion with about interconnection rights to",
 'insert appropriate payment',
 'forward this to max',
 'reply to']

..
..


In [34]:
from nltk.corpus import words

In [37]:
len(words.words())

236736

In [38]:
import nltk

In [42]:
for w in nltk.corpus.wordnet.words(): print(w)

.22-caliber
.22-calibre
.22_caliber
.22_calibre
.38-caliber
.38-calibre
.38_caliber
.38_calibre
.45-caliber
.45-calibre
.45_caliber
.45_calibre
0
1
10
10-membered
100
1000
1000th
100th
101
101st
105
105th
10th
11
110
110th
115
115th
11th
12
120
120th
125
125th
12th
13
130
130th
135
135th
13th
14
140
140th
145
145th
14th
15
150
150th
155
155th
15th
16
160
160th
165
165th
16th
17
170
170th
175
175th
17th
18
180
180th
18th
19
190
190th
19th
1st
2
2-dimensional
20
200
200th
20th
21
21st
22
22nd
23
23rd
24
24th
25
25th
26
26th
27
27th
28
28th
29
29th
2d
2nd
3
3-dimensional
3-membered
30
300
300th
30th
31
31st
32
32nd
33
33rd
34
34th
35
35th
36
36th
37
37th
38
38th
39
39th
3rd
4
4-dimensional
4-membered
40
400
400th
40th
41
41st
42
42nd
43
43rd
44
44th
45
45th
46
46th
47
47th
48
48th
49
49th
4th
5
5-membered
50
500
500th
50th
51
52
53
54
55
55th
56
57
58
59
5th
6
6-membered
60
60th
61
62
63
64
64th
65
65th
66
67
68
69
6th
7
7-membered
70
70th
71
72
73
74
75
75th
76
77
78
79
7th
8
8-membered


biform
bifurcate
bifurcated
big
big-bellied
big-boned
big-chested
big-shouldered
big-ticket
bigamous
bigeminal
bigeneric
bigger
biggish
bigheaded
bighearted
bigmouthed
bignoniaceous
bigoted
bilabial
bilabiate
bilateral
bilaterally_symmetric
bilaterally_symmetrical
bilgy
biliary
bilinear
bilingual
bilious
billed
billiard
billion
billionth
billowing
billowy
bilobate
bilobated
bilobed
bilocular
biloculate
bimanual
bimestrial
bimetal
bimetallic
bimetallistic
bimillenial
bimodal
bimolecular
bimonthly
bimorphemic
bimotored
binary
binate
binaural
bindable
binding
binocular
binomial
binominal
binuclear
binucleate
binucleated
biocatalytic
biochemical
bioclimatic
biodegradable
biogenetic
biogenic
biogenous
biogeographic
biogeographical
biographic
biographical
biologic
biological
biologistic
bioluminescent
biomedical
bionic
bionomic
bionomical
biosynthetic
biosystematic
biotic
biotitic
biotypic
biovular
biparous
bipartisan
bipartite
bipartizan
biped
bipedal
bipinnate
bipinnatifid
bipolar
biquadra

cracking
crackle
crafty
cragfast
cragged
craggy
cram_full
cramped
cranial
craniometric
craniometrical
crank
cranky
crannied
crappy
crapulent
crapulous
crashing
crass
craved
craven
crazed
crazy
creaky
cream-colored
creamy
creamy-colored
creamy-white
creamy-yellow
crease-resistant
creaseless
creaseproof
creative
credal
credentialled
credible
creditable
credited
creditworthy
credulous
creedal
creepy
creepy-crawly
crenate
crenated
crenulate
crenulated
creole
crepuscular
crescendo
crescent
crescent-shaped
cress_green
cresson
crested
crestfallen
cretaceous
cretinous
criminal
criminative
criminatory
criminological
crimson
crimson-magenta
crimson-purple
crimson-yellow
cringing
crinkled
crinkly
crinoid
crippled
crippling
crisp
crispate
crispy
crisscross
crisscrossed
criterial
criterional
critical
croaky
croatian
crocked
crocketed
cromwellian
crookback
crookbacked
crooked
cropped
cross
cross-banded
cross-cultural
cross-eyed
cross-grained
cross-linguistic
cross-modal
cross-ply
cross-pollinating
c

fugly
fulfilled
fulgent
fulgid
fulgurant
fulgurating
fulgurous
full
full-blood
full-blooded
full-blown
full-bodied
full-bosomed
full-clad
full-dress
full-face
full-fashioned
full-fledged
full-grown
full-length
full-page
full-scale
full-size
full-strength
full-term
full-time
full_of_life
fully_fashioned
fully_fledged
fully_grown
fulminant
fulsome
fumbling
fumed
functional
functionally_illiterate
functioning
fundamental
fundamentalist
fundamentalistic
funded
funerary
funereal
fungal
fungible
fungicidal
fungoid
fungous
funguslike
funicular
funky
funnel-shaped
funny
fur-bearing
furious
furled
furlike
furnished
furred
furrowed
furry
further
furthermost
furthest
furtive
fuscous
fused
fusible
fusiform
fussy
fusty
futile
future
future_day
futureless
futurist
futuristic
fuzzed
fuzzy
gabby
gabled
gabonese
gaelic
gaelic-speaking
gaga
gainful
gainly
galactic
galilaean
galilean
gallant
galled
gallic
gallican
gallinaceous
galling
galore
galvanic
galvanising
galvanizing
galwegian
gambian
game
gamey
g

lowly
lowset
loyal
lubberly
lubricated
lubricious
lucent
lucid
lucifugal
lucifugous
luckless
lucky
lucrative
luculent
lucullan
ludicrous
lugubrious
lukewarm
lumbar
lumbering
lumbosacral
luminescent
luminous
lumpen
lumpish
lumpy
lunar
lunate
lunatic
lung-like
lunisolar
lupine
lurid
luscious
lush
lusitanian
lusterless
lustful
lustreless
lustrous
lusty
luteal
lutheran
luxe
luxembourgian
luxemburger
luxuriant
luxurious
lv
lvi
lvii
lviii
lx
lxi
lxii
lxiii
lxiv
lxv
lxvi
lxvii
lxviii
lxx
lxxi
lxxii
lxxiii
lxxiv
lxxv
lxxvi
lxxvii
lxxviii
lxxx
lxxxi
lxxxii
lxxxiii
lxxxiv
lxxxv
lxxxvi
lxxxvii
lxxxviii
lying_in_wait
lymphatic
lymphoblast-like
lymphocytic
lymphoid
lynx-eyed
lyonnaise
lyophilised
lyophilized
lyrate
lyre-shaped
lyric
lyrical
lysogenic
m
macabre
macaronic
macedonian
macerative
machiavellian
machine-accessible
machine-controlled
machine-driven
machine-made
machine_readable
machinelike
macho
macrencephalic
macrencephalous
macro
macrobiotic
macrocephalic
macrocephalous
macrocosmic
macro

...

viola_pubescens
viola_reichenbachiana
viola_rostrata
viola_striata
viola_sylvatica
viola_tricolor
viola_tricolor_hortensis
violaceae
violation
violator
violence
violent_death
violent_disorder
violent_storm
violent_stream
violet-flowered_petunia
violet_family
violet_suksdorfia
violet_wood_sorrel
violin
violin_bow
violin_family
violin_lesson
violin_maker
violin_section
violinist
violist
violoncellist
violoncello
viomycin
viorna_baldwinii
viosterol
vioxx
vip
viper
viper's_bugloss
viper's_grass
vipera
vipera_aspis
vipera_berus
viperidae
viperine_grass_snake
viracept
viraemia
virago
viral_delivery_vector
viral_haemorrhagic_fever
viral_hemorrhagic_fever
viral_hepatitis
viral_infection
viral_pneumonia
viramune
virazole
virchow
viremia
vireo
vireo_olivaceous
vireo_solitarius
vireo_solitarius_solitarius
vireonidae
virga
virgil
virgil_garnett_thomson
virgil_thomson
virgilia
virgilia_capensis
virgilia_divaricata
virgilia_oroboides
virgin's_bower
virgin_birth
virgin_forest
virgin_islands
virgin_is

witches'_sabbath
witchgrass
withdrawal
withdrawal_method
withdrawal_symptom
withdrawer
withdrawing_room
withdrawnness
withe
withers
witherspoon
withholder
withholding
withholding_tax
withstander
withy
witloof
witness
witness_box
witness_stand
witnesser
witold_gombrowicz
wits
wittgenstein
witticism
wittiness
wittol
witwatersrand
wivern
wiz
wizardry
wlan
wmd
wmo
wnw
woad
woadwaxen
wobble
wobbler
wodan
wodehouse
woden
woe
woefulness
wog
wok
wold
wolf
wolf's_bane
wolf's_milk
wolf_bean
wolf_boy
wolf_cub
wolf_fish
wolf_pack
wolf_pup
wolf_spider
wolfbane
wolfe
wolff
wolffia
wolffia_columbiana
wolffiella
wolffiella_gladiata
wolffish
wolfgang_amadeus_mozart
wolfgang_pauli
wolfhound
wolfman
wolfram
wolfram_steel
wolframite
wolfsbane
wollaston
wollaston_prism
wollastonite
wollemi_pine
wollstonecraft
wolof
wolstonian_glaciation
wolverine
wolverine_state
woman
woman's_body
woman's_clothing
woman's_doctor
woman's_hat
woman-worship
woman_chaser
woman_hater
woman_of_the_house
woman_of_the_street
woman

contract_in
contract_out
contradict
contradistinguish
contraindicate
contravene
contribute
contrive
controvert
contuse
convalesce
convect
convene
conventionalise
conventionalize
converge
convey
convince
convoke
convolve
convulse
cooccur
cooccur_with
cook_out
cook_up
cool_down
cool_it
cool_off
cool_one's_heels
coop_in
coop_up
cooperate
cop_out
cope_with
copolymerise
copolymerize
copper-bottom
copulate
copy_out
copyedit
copyread
coquet
cordon_off
core_out
cork_up
coronate
corrade
correspond
corroborate
corrode
corrugate
coruscate
cosign
cosponsor
cosset
cotton_on
cotton_up
cough_out
cough_up
count_down
count_off
count_on
count_out
counter-drill
counteract
counterchallenge
counterchange
counterpose
counterstrike
countervail
couple_on
couple_up
court_favor
court_favour
cover_for
cover_up
covet
cower
cozen
cozy_up
crack_down
crack_up
cram
cranch
crank_out
crank_up
crap_up
crash-dive
crash_land
craunch
crave
crawfish_out
crawl_in
cream_off
create
create_by_mental_act
create_from_raw_material

hypothesize
ice_over
ice_up
idealise
idealize
ideate
identify
idolise
idolize
ignite
ignore
ill-treat
ill-use
illegalise
illegalize
illume
illuminate
illumine
illustrate
imagine
imbed
imbibe
imbrue
imbue
imitate
immaterialise
immaterialize
immerse
immigrate
immingle
immix
immobilise
immobilize
immolate
immortalise
immortalize
immunise
immunize
immure
impair
impale
impanel
impart
impeach
impede
impel
impend
imperil
impersonate
impinge
impinge_on
implicate
implode
implore
imply
importune
impose
impound
impoverish
imprecate
impregnate
imprison
improve
improvise
improvize
impugn
impulse-buy
impute
inactivate
inaugurate
incandesce
incapacitate
incarcerate
incarnadine
incase
incinerate
incise
incite
inclose
include
incommode
inconvenience_oneself
incriminate
incrust
incubate
inculcate
inculpate
incur
indemnify
indicate
indict
indispose
indite
individualise
individualize
individuate
indoctrinate
indorse
induce
induct
indue
indulge
industrialise
industrialize
indwell
infatuate
infect
infer
inf

shake_up
shamanise
shamanize
shape_up
sharpen
sharpshoot
shatter
she-bop
sheathe
shed_blood
shed_light_on
shell_out
shellack
shelve
shew
shine_at
shine_up
shirk
shirr
shmoose
shnorr
shoo
shoo_away
shoo_off
shoot_a_line
shoot_craps
shoot_down
shoot_for
shoot_one's_mouth_off
shoot_the_breeze
shoot_up
shop_at
shoplift
shore_up
short-change
short-circuit
shorten
shoulder_in
shout_down
shout_out
shove_along
shove_off
shovel_in
show_off
show_the_door
show_up
shower_down
shrink_back
shrink_from
shrinkwrap
shrive
shrivel
shrivel_up
shrug_off
shun
shush
shut_away
shut_down
shut_in
shut_off
shut_one's_mouth
shut_out
shy_away_from
sibilate
sicken
side-slip
sidle
sidle_up
sieve_out
sift
sight-read
sight-sing
sightread
sightsee
sightsing
sign_away
sign_in
sign_off
sign_on
sign_over
sign_up
signalise
signalize
signify
silt_up
silver-plate
silverplate
simmer_down
simonise
simonize
simplify
simulate
sing
sing_along
single_out
singularise
singularize
sink_in
sinter
siphon_off
siss
sit
sit_around
sit_ba

In [44]:
len(list(nltk.corpus.wordnet.words()))

147306